# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

In [16]:
import xlwings as xw
import numpy as np

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [8]:
wb = xw.Book('data/себестоимостьА_в1.xlsx')

In [9]:
sheet = wb.sheets['Рецептура']

In [18]:
#sheet.range('G7:O10').value
consumption = sheet.range('G7:O10').options(np.array).value
consumption

array([[0.2  , 0.4  , 0.02 , 0.02 , 0.01 , 0.005, 0.1  ,   nan,   nan],
       [0.07 , 0.25 , 0.01 , 0.06 , 0.015,   nan, 0.06 , 0.05 ,   nan],
       [0.12 , 0.22 , 0.005,   nan, 0.009, 0.005,   nan, 0.05 , 0.03 ],
       [0.12 , 0.23 , 0.01 ,   nan, 0.015,   nan,   nan,   nan, 0.05 ]])

In [19]:
unit_price = sheet.range('G14:O14').options(np.array).value
unit_price

array([  7.,  10., 184.,  19.,   7., 290., 105.,  83., 260.])

In [33]:
values = np.nan_to_num(consumption+unit_price).sum(axis=1).reshape(4,1)

In [40]:
values

array([[622.755],
       [415.515],
       [841.439],
       [468.425]])

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [24]:
#sheet.range('T7:T10').options(tranpose=True).value = np.nan_to_num(consumption*unit_price).sum(axis=1)

In [41]:
sheet.range('T7:T10').value = values

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [42]:
sheet.range('T6').value = 'Себестоимость'

In [45]:
sheet.range('T4:T6').api.Merge()

4. Выполнить 3 с помощью "протягиваемых" формул.

In [53]:
from xlwings.constants import AutoFillType

In [54]:
# =СУММПРОИЗВ(G7:O7;$G$14:$O$14)
sheet.range("V7").formula = '=SUMPRODUCT(G7:O7,$G$14:$O$14)'
sheet.range("V7").api.AutoFill(sheet.range('V7:V10').api,AutoFillType.xlFillDefault)

True

## Лабораторная работа 7

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [106]:
import pandas as pd
import numpy as np

In [107]:
reviews = pd.read_csv('data/reviews_sample.csv',sep = ',', header=0).rename(columns={"Unnamed: 0": 'id'})
recipes = pd.read_csv('data/recipes_sample_with_tags_ingredients.csv',sep = ',',parse_dates=['submitted'])[['id','name','minutes','submitted','description','n_ingredients']]

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [108]:
reviews = reviews.sample(frac=0.05)
recipes = recipes.sample(frac=0.05)

In [109]:
with pd.ExcelWriter('data/recipes.xlsx') as writer:  
    recipes.to_excel(writer,sheet_name='Рецепты')
    reviews.to_excel(writer,sheet_name='Отзывы')

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [110]:
import xlwings as xw
wb = xw.Book('data/recipes.xlsx')
sheet = wb.sheets['Рецепты']

In [111]:
sheet.range('H1').value = 'seconds_assign'
minutes = sheet.range('D2:D1501').options(np.array).value.reshape(1500,1)
seconds = minutes * 60
sheet.range('H2:H1501').value = seconds

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [112]:
sheet.range('I1').value = 'seconds_formula'

In [113]:
from xlwings.constants import AutoFillType
sheet.range("I2").formula = '=PRODUCT(D2,60)'
sheet.range("I2").api.AutoFill(sheet.range('I2:I1501').api,AutoFillType.xlFillDefault)

True

5. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [114]:
sheet.range('J1').value = 'n_reviews'
sheet.range("J2").formula = '=COUNTIF(Отзывы!D$2:D6336,$B2)'
sheet.range("J2").api.AutoFill(sheet.range('J2:J1501').api,AutoFillType.xlFillDefault)

True

6. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [60]:
sheet.range('H1:J1').api.Font.Bold = True
sheet.range('H1:J1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

7. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [63]:
for i in sheet.range('D2:D1501'):
    value = i.value
    if(value>10):
        i.api.Interior.ColorIndex = 3
    elif(value<5):
        i.api.Interior.ColorIndex = 4
    elif(value>=5 and value<=10):
        i.api.Interior.ColorIndex = 6

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [104]:
def validate():
    sheet_recipes = wb.sheets['Рецепты']
    sheet_reviews = wb.sheets['Отзывы']
    recipes = np.array(sheet_recipes.range('B2:B1501').value)
    for i in range(2,6337):
        row = sheet_reviews.range(f'B{i}:G{i}').value
        if(row[4]>=0 and row[4]<=5 and (row[2] in recipes)):
            continue
        else:
            sheet_reviews.range(f'B{i}:G{i}').api.Interior.ColorIndex = 3
    return

In [105]:
validate()